<a href="https://colab.research.google.com/github/Lauralug0/GBC/blob/main/LauraLugo_Task3_News_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [3]:
!pip install pyLDAvis gensim spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 71.6 MB/s eta 0:00:00


### Import the libraries

In [4]:
import json
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

--2021-03-08 08:56:47--  https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... 

/Users/subashgandyer/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


connected.
HTTP request sent, awaiting response... 200 OK
Length: 23237087 (22M) [text/plain]
Saving to: ‘newsgroups.json’

newsgroups.json     100%[===================>]  22.16M  13.8MB/s    in 1.6s    

2021-03-08 08:56:49 (13.8 MB/s) - ‘newsgroups.json’ saved [23237087/23237087]



### Load the dataset

In [6]:
with open('newsgroups.json', 'r') as f:
    data_loaded = json.load(f)

data = [re.sub(r'\n', '', entry_text) for entry_text in data_loaded['content'].values()]

In [7]:
print(data[:1])

["From: lerxst@wam.umd.edu (where's my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL   ---- brought to you by your neighborhood Lerxst ----"]


### Preprocess the data

### Email Removal

In [8]:
data_no_emails = [re.sub(r'\S*@\S*\s?', '', entry_text) for entry_text in data]

In [9]:
print(data_no_emails[:1])

["From: (where's my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL   ---- brought to you by your neighborhood Lerxst ----"]


### Newline Removal

In [11]:
data_no_newlines = [re.sub('\s+', ' ', entry_text) for entry_text in data_no_emails]
print(data_no_newlines[:1])

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1788968781.py:1: SyntaxWarning: invalid escape sequence '\s'
  data_no_newlines = [re.sub('\s+', ' ', entry_text) for entry_text in data_no_emails]


["From: (where's my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL ---- brought to you by your neighborhood Lerxst ----"]


### Single Quotes Removal

In [12]:
data_no_quotes = [re.sub("'", '', entry_text) for entry_text in data_no_newlines]

### Tokenize
- Create **sent_to_words()**
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [14]:
data_words = list(sent_to_words(data_no_quotes))
print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'eduorganization', 'university', 'of', 'maryland', 'college', 'parklines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'sawthe', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'yearsof', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'youhave', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [17]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### remove_stopwords( )

In [18]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [19]:
data_words_no_stopwords = remove_stopwords(data_words)
print(data_words_no_stopwords[:1])

[['wheres', 'thing', 'car', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'eduorganization', 'university', 'maryland', 'college', 'parklines', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'sawthe', 'day', 'door', 'sports', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'yearsof', 'production', 'car', 'made', 'history', 'whatever', 'info', 'youhave', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst']]


### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

In [20]:

bigram = gensim.models.Phrases(data_words_no_stopwords, min_count=5, threshold=100)

In [22]:

bigram_mod = gensim.models.phrases.Phraser(bigram)

#### make_bigrams( )

In [25]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [26]:
data_words_bigrams = make_bigrams(data_words_no_stopwords)

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [27]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#### lemmatizaton( )

In [31]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [32]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [33]:
print(data_lemmatized[:1])

[['s', 'thing', 'car', 'nntp_poste', 'wam_umd', 'eduorganization', 'university', 'parkline', 'wonder', 'enlighten', 'car', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'yearsof', 'production', 'car', 'make', 'history', 'info', 'youhave', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


### Create a Dictionary

In [35]:
id2word = corpora.Dictionary(data_lemmatized)

### Create Corpus

In [36]:
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

### Filter low-frequency words

In [37]:

id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

corpus = [id2word.doc2bow(text) for text in data_lemmatized]

print(f"Number of unique tokens after filtering: {len(id2word)}")
print(f"First 10 entries of the filtered corpus: {corpus[:10]}")

Number of unique tokens after filtering: 5946
First 10 entries of the filtered corpus: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)], [(3, 1), (5, 2), (21, 1), (31, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1)], [(5, 2), (12, 2), (13, 1), (15, 2), (17, 1), (24, 2), (26, 1), (31, 1), (34, 1), (39, 1), (51, 1), (55, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 2), (84, 3), (85, 1), (8

### Create Index 2 word dictionary

In [42]:

print('Sample of (id, word) mapping:')
for i in range(min(5, len(id2word))):
    print(f'  {i}: {id2word[i]}')

Sample of (id, word) mapping:
  0: addition
  1: body
  2: bring
  3: call
  4: car


In [43]:
print(id2word.token2id)
print(id2word)

{'addition': 0, 'body': 1, 'bring': 2, 'call': 3, 'car': 4, 'day': 5, 'door': 6, 'early': 7, 'eduorganization': 8, 'engine': 9, 'enlighten': 10, 'history': 11, 'info': 12, 'know': 13, 'late': 14, 'look': 15, 'mail': 16, 'make': 17, 'model': 18, 'name': 19, 'neighborhood': 20, 'nntp_poste': 21, 'parkline': 22, 'production': 23, 'really': 24, 'rest': 25, 's': 26, 'separate': 27, 'small': 28, 'spec': 29, 'sport': 30, 'thank': 31, 'thing': 32, 'university': 33, 'wonder': 34, 'youhave': 35, 'acceleration': 36, 'adapter': 37, 'add': 38, 'answer': 39, 'attain': 40, 'base': 41, 'brave': 42, 'brief': 43, 'card': 44, 'clock': 45, 'cpu': 46, 'do': 47, 'especially': 48, 'experience': 49, 'fair': 50, 'final': 51, 'floppy': 52, 'hour': 53, 'message': 54, 'next': 55, 'number': 56, 'oscillator': 57, 'poll': 58, 'procedure': 59, 'rate': 60, 'request': 61, 'send': 62, 'si': 63, 'soul': 64, 'speed': 65, 'summarize': 66, 'top': 67, 'university_washingtonline': 68, 'upgrade': 69, 'usage': 70, 'access': 71,

### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [47]:
lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    num_topics=20,
    random_state=100,
    chunksize=2000,
    passes=10,
    alpha='asymmetric',
    per_word_topics=True
)

### Print the Keyword in the 10 topics

In [48]:
print(lda_model.print_topics(num_words=10))

[(0, '0.014*"nntp_poste" + 0.013*"host" + 0.013*"get" + 0.013*"price" + 0.013*"good" + 0.011*"buy" + 0.011*"m" + 0.010*"sell" + 0.010*"look" + 0.010*"new"'), (1, '0.017*"car" + 0.013*"get" + 0.009*"article" + 0.009*"go" + 0.008*"bike" + 0.007*"make" + 0.007*"well" + 0.006*"good" + 0.006*"time" + 0.006*"much"'), (2, '0.016*"problem" + 0.015*"card" + 0.013*"get" + 0.012*"use" + 0.011*"driver" + 0.011*"window" + 0.011*"know" + 0.010*"bit" + 0.010*"run" + 0.010*"work"'), (3, '0.043*"key" + 0.014*"encryption" + 0.012*"system" + 0.012*"use" + 0.010*"chip" + 0.010*"public" + 0.010*"security" + 0.009*"bit" + 0.009*"government" + 0.007*"secure"'), (4, '0.030*"article" + 0.014*"organization" + 0.012*"get" + 0.010*"say" + 0.010*"know" + 0.010*"go" + 0.009*"m" + 0.009*"day" + 0.008*"take" + 0.008*"never"'), (5, '0.022*"play" + 0.020*"game" + 0.018*"player" + 0.014*"team" + 0.013*"hockey" + 0.012*"go" + 0.011*"good" + 0.011*"year" + 0.011*"point" + 0.011*"get"'), (6, '0.052*"drive" + 0.023*"file" +

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [49]:

print('Perplexity: ', lda_model.log_perplexity(corpus))

Perplexity:  -7.350432941893165


### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [50]:

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.515117898286804


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [51]:
import pyLDAvis
import pyLDAvis.gensim_models as gensim_models

pyLDAvis.enable_notebook()
vis = gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.054847 -0.076475       1        1  8.877871
19    -0.072725  0.125728       2        1  8.513954
17    -0.099554  0.028169       3        1  8.061468
16     0.236893 -0.038974       4        1  7.878395
2      0.097843 -0.134978       5        1  7.208239
0      0.028881 -0.129171       6        1  6.841330
15    -0.160780 -0.018323       7        1  5.701588
13     0.095839  0.169181       8        1  5.302424
7     -0.068691  0.163630       9        1  4.525304
3      0.118844  0.065184      10        1  4.499364
4     -0.122860 -0.034864      11        1  4.262943
9     -0.091133  0.018776      12        1  4.060629
12    -0.068167  0.170073      13        1  3.970793
10    -0.083803 -0.023713      14        1  3.880718
5     -0.118709 -0.127702      15        1  3.496107
14    -0.006145  0.070165      16        1  2.932743
18    -0.148374 -0.145889      17        1  2.641628
6      0.155620 -0.127984      18        1  2.618537
8      0.188515  0.043280      19        1  2.509635
11     0.173355  0.003888      20        1  2.216329, topic_info=       Term         Freq        Total Category  logprob  loglift
285    file  2285.000000  2285.000000  Default  30.0000  30.0000
572     key  1905.000000  1905.000000  Default  29.0000  29.0000
791   drive  1852.000000  1852.000000  Default  28.0000  28.0000
1066   game  1592.000000  1592.000000  Default  27.0000  27.0000
1374    gun  1204.000000  1204.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
16     mail   136.133410  1723.295304  Topic20  -5.0507   1.2710
249    need   154.843316  3153.729360  Topic20  -4.9219   0.7954
280    year   145.382529  3014.652171  Topic20  -4.9850   0.7774
124    post   136.027744  2202.267631  Topic20  -5.0515   1.0249
148     use   112.111370  4573.705477  Topic20  -5.2448   0.1007

[1501 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3681      1  0.016285        _
3681      2  0.227994        _
3681      4  0.138425        _
3681     13  0.606627        _
3681     20  0.004071        _
...     ...       ...      ...
280      17  0.137993     year
280      19  0.061035     year
280      20  0.048098     year
2860     11  0.976253    yeast
4562      9  0.976947  zionist

[5969 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 20, 18, 17, 3, 1, 16, 14, 8, 4, 5, 10, 13, 11, 6, 15, 19, 7, 9, 12])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
